In [4]:
from astropy.coordinates import SkyCoord
from astropy import units as u

data = [
    ("ZTF17aabtvsy", "SN2022yei", "10:35:32.09", "+37:38:59.0"),
    ("ZTF17aabvong", "SN2024xxq", "02:05:07.68", "+11:14:55.1"),
    ("ZTF17aacldgo", "SN2022zxv", "03:09:24.35", "-04:53:39.2"),
    ("ZTF17aadlxmv", "SN2020adv", "08:29:47.59", "+33:54:22.8"),
    ("ZTF18aaaonon", "SN2022jjs", "10:19:05.51", "+14:24:16.6"),
]

def print_coords_in_degrees(data):
    for ztfid, iauid, ra, dec in data:
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
        print(f"ZTFID: {ztfid}, IAUID: {iauid}, RA: {coord.ra.deg:.6f}, Dec: {coord.dec.deg:.6f}")

print_coords_in_degrees(data)


ZTFID: ZTF17aabtvsy, IAUID: SN2022yei, RA: 158.883708, Dec: 37.649722
ZTFID: ZTF17aabvong, IAUID: SN2024xxq, RA: 31.282000, Dec: 11.248639
ZTFID: ZTF17aacldgo, IAUID: SN2022zxv, RA: 47.351458, Dec: -4.894222
ZTFID: ZTF17aadlxmv, IAUID: SN2020adv, RA: 127.448292, Dec: 33.906333
ZTFID: ZTF18aaaonon, IAUID: SN2022jjs, RA: 154.772958, Dec: 14.404611


In [9]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.sdss import SDSS

# Read the CSV file
try:
    df = pd.read_csv("/home/jovyan/research/2.14.25/diego_data.csv")  # Replace with your actual file name
except FileNotFoundError:
    print("Error: CSV file not found. Please make sure the file exists in the correct directory.")
    exit()

# Iterate through the first 5 supernovae
for i in range(min(5, len(df))):  # Limit to 5 or the number of rows if less than 5
    ztf_id = df['ZTFID'][i]  # Correct column name for ZTF ID
    ra_dec_str = f"{df['RA'][i]} {df['Dec'][i]}"  # Combine RA and Dec into one string

    try:
        # Use SkyCoord to parse the RA and Dec string directly
        sn_coords = SkyCoord(ra_dec_str, unit=(u.hourangle, u.deg)) #Handles HH:MM:SS and DD:MM:SS

        search_radius = 60 * u.arcsec

        results = SDSS.query_region(
            sn_coords,
            radius=search_radius,
            data_release=17,
            photoobj_fields=['objID', 'ra', 'dec', 'type', 'specObjID', 'z', 'u', 'g', 'r', 'i'],
        )

        print(f"\nZTF ID: {ztf_id}")

        if results is not None and len(results) > 0:
            distances = sn_coords.separation(SkyCoord(ra=results['ra'] * u.degree, dec=results['dec'] * u.degree, frame='icrs'))
            sorted_indices = distances.argsort()
            sorted_results = results[sorted_indices]

            galaxies = sorted_results[sorted_results['type'] == 3]

            if len(galaxies) > 0:
                closest_galaxy = galaxies[0]
                print("Possible Host Galaxy:")
                print(f"  SDSS ID: {closest_galaxy['objID']}")
                print(f"  Type: {closest_galaxy['type']}")
                print(f"  RA: {closest_galaxy['ra']}")
                print(f"  Dec: {closest_galaxy['dec']}")
                print(f"  g: {closest_galaxy['g']}")
                print(f"  r: {closest_galaxy['r']}")
                print(f"  i: {closest_galaxy['i']}")
                print(f"  z: {closest_galaxy['z']}")
                print(f"  u: {closest_galaxy['u']}")
                
                print("CHECK THE WEBSITE TO CONFIRM")
            else:
                print("No galaxies found within the search radius.")

        else:
            print("No SDSS objects found within the search radius.")

    except (ValueError, AttributeError) as e:
        print(f"Error processing coordinates for ZTF ID {ztf_id}: {e}")
        continue  # Skip to the next supernova if there's an error


ZTF ID: ZTF17aabtvsy
Possible Host Galaxy:
  SDSS ID: 1237662225669161128
  Type: 3
  RA: 158.883596183724
  Dec: 37.6495783499432
  g: 16.04441
  r: 15.19806
  i: 14.7865
  z: 14.47726
  u: 17.92454
CHECK THE WEBSITE TO CONFIRM

ZTF ID: ZTF17aabvong
Possible Host Galaxy:
  SDSS ID: 1237678919681179839
  Type: 3
  RA: 31.2821072265924
  Dec: 11.2485784557871
  g: 15.56673
  r: 14.88181
  i: 14.50039
  z: 14.19007
  u: 16.86292
CHECK THE WEBSITE TO CONFIRM

ZTF ID: ZTF17aacldgo
No SDSS objects found within the search radius.

ZTF ID: ZTF17aadlxmv
Possible Host Galaxy:
  SDSS ID: 1237657628436005116
  Type: 3
  RA: 127.448173699413
  Dec: 33.9064715077163
  g: 16.20895
  r: 15.49266
  i: 15.10928
  z: 14.81678
  u: 17.73309
CHECK THE WEBSITE TO CONFIRM

ZTF ID: ZTF18aaaonon
Possible Host Galaxy:
  SDSS ID: 1237671123232292959
  Type: 3
  RA: 154.773033886085
  Dec: 14.4047321037697
  g: 17.31716
  r: 16.39745
  i: 15.94811
  z: 15.60975
  u: 19.31708
CHECK THE WEBSITE TO CONFIRM


In [6]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.sdss import SDSS

# Input parameters
ra = 158.883596183724
dec = 37.6495783499432
sdss_id = 1237662225669161128  # Keeping this for reference

# Create SkyCoord object
coordinates = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

try:
    results = SDSS.query_region(coordinates, radius="0.001 deg", 
                                 data_release=17,  # Specify data release!
                                 fields=['z', 'specObjID', 'class'])  # Only 'class' here

    if results:
        redshift = results[0]['z']
        specobj_id = results[0]['specObjID']
        obj_class = results[0]['class']

        print(f"Redshift (z): {redshift}")
        print(f"SpecObjID: {specobj_id}")
        print(f"Class: {obj_class}")


        # Get subclass using specObjID (requires a separate query)
        if specobj_id is not None:  # Check if specObjID was returned
            spec_results = SDSS.get_spectrum(specObjID=specobj_id, data_release=17)  # Correct way to get subclass
            if spec_results:
                obj_subclass = spec_results[0].header.get('SUBCLASS')  # Extract from FITS header
                print(f"Subclass: {obj_subclass}")
                print(f"Source Type: {obj_class} - {obj_subclass}")
            else:
                print("Could not retrieve spectrum to determine subclass.")
        else:
            print("Could not retrieve specObjID.")


    else:
        print("No matching object found in SDSS.")

except Exception as e:
    print(f"An error occurred during the SDSS query: {e}")

Redshift (z): 14.47726
SpecObjID: 2221570998082234368
Class: GALAXY
An error occurred during the SDSS query: 'SDSSClass' object has no attribute 'get_spectrum'


In [21]:
from astroquery.sdss import SDSS
from astropy import units as u
from astropy.coordinates import SkyCoord

ra = 158.883596183724
dec = 37.6495783499432

try:
    coordinates = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

    results = SDSS.query_region(coordinates, radius="0.0003 deg",  # Very small radius for single object
                                data_release=17,
                                fields=['z', 'u', 'g', 'r', 'i', 'z', 'class', 'specObjID']) # Include specObjID

    if results:
        galaxy_data = results[0]

        redshift = galaxy_data['z']
        print(f"Redshift (z): {redshift}")

        magnitudes = {
            'u': galaxy_data['u'],
            'g': galaxy_data['g'],
            'r': galaxy_data['r'],
            'i': galaxy_data['i'],
            'z': galaxy_data['z'],
        }

        print("Magnitudes:")
        for filter_name, magnitude in magnitudes.items():
            print(f"{filter_name}-band: {magnitude}")

        specObjID = galaxy_data['specObjID'] # Get specObjID from the query

        # Get spectral data (using specObjID)
        if specObjID is not None:
            spec_results = SDSS.get_spectrum(specObjID=specObjID, data_release=17)
            if spec_results:
                obj_subclass = spec_results[0].header.get('SUBCLASS')
                obj_class = galaxy_data['class']
                print(f"Subclass: {obj_subclass}")
                print(f"Source Type: {obj_class} - {obj_subclass}")
            else:
                print("Could not retrieve spectrum to determine subclass.")
        else:
            print("Could not retrieve specObjID.")

    else:
        print("No matching object found in SDSS.")

except Exception as e:
    print(f"An error occurred during the SDSS query: {e}")

Redshift (z): 14.47726
Magnitudes:
u-band: 17.92454
g-band: 16.04441
r-band: 15.19806
i-band: 14.7865
z-band: 14.47726
An error occurred during the SDSS query: 'SDSSClass' object has no attribute 'get_spectrum'


In [23]:
from astroquery.sdss import SDSS
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np

ra = 158.883596183724
dec = 37.6495783499432

try:
    coordinates = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

    results = SDSS.query_region(coordinates, radius="0.0003 deg",
                                data_release=17,
                                fields=['z', 'u', 'g', 'r', 'i', 'z', 'class', 'specObjID', 'zErr'])

    if results:
        galaxy_data = results[0]

        redshift = galaxy_data['z']
        z_err = galaxy_data['zErr']

        if z_err > 0:
            print(f"Redshift (z): {redshift}")
        else:
            print("No spectroscopic redshift available.")

        magnitudes = {
            'u': galaxy_data['u'],
            'g': galaxy_data['g'],
            'r': galaxy_data['r'],
            'i': galaxy_data['i'],
            'z': galaxy_data['z'],
        }

        print("Magnitudes:")
        for filter_name, magnitude in magnitudes.items():
            print(f"{filter_name}-band: {magnitude}")

        specObjID = galaxy_data['specObjID']

        if specObjID is not None:
            # Get spectral data (using get_spectra - plural!)
            spectra = SDSS.get_spectra(plate=galaxy_data['plate'],
                                      mjd=galaxy_data['mjd'],
                                      fiberID=galaxy_data['fiberID'],
                                      data_release=17)
            if spectra:
                spectrum = spectra[0]  # Access the first spectrum (should be only one)
                obj_subclass = spectrum.header.get('SUBCLASS')
                obj_class = galaxy_data['class']
                print(f"Subclass: {obj_subclass}")
                print(f"Source Type: {obj_class} - {obj_subclass}")

                # Example of accessing spectral data (flux and wavelength):
                # flux = spectrum.data['flux']
                # wavelength = spectrum.data['wavelength']
                # You can now work with the flux and wavelength data
            else:
                print("Could not retrieve spectrum.")
        else:
            print("Could not retrieve specObjID.")

    else:
        print("No matching object found in SDSS.")

except Exception as e:
    print(f"An error occurred during the SDSS query: {e}")

Redshift (z): 14.47726
Magnitudes:
u-band: 17.92454
g-band: 16.04441
r-band: 15.19806
i-band: 14.7865
z-band: 14.47726
An error occurred during the SDSS query: 'plate'


In [24]:
from astroquery.sdss import SDSS
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np

ra = 158.883596183724
dec = 37.6495783499432

try:
    coordinates = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

    results = SDSS.query_region(coordinates, radius="0.0003 deg",
                                data_release=17,
                                fields=['z', 'u', 'g', 'r', 'i', 'class', 'specObjID', 'zErr'])

    if results:
        galaxy_data = results[0]

        redshift = galaxy_data['z']
        z_err = galaxy_data['zErr']

        if z_err > 0:
            print(f"Redshift (z): {redshift}")
        else:
            print("No spectroscopic redshift available.")

        magnitudes = {
            'u': galaxy_data['u'],
            'g': galaxy_data['g'],
            'r': galaxy_data['r'],
            'i': galaxy_data['i'],
            'z': galaxy_data['z'],
        }

        print("Magnitudes:")
        for filter_name, magnitude in magnitudes.items():
            print(f"{filter_name}-band: {magnitude}")

        specObjID = galaxy_data['specObjID']

        if specObjID is not None:
            # 1. Query specObj table to get plate, mjd, fiberID
            specobj_query = f"SELECT plate, mjd, fiberID FROM specObj WHERE specObjID = {specObjID}"
            specobj_results = SDSS.query(specobj_query, data_release=17)

            if specobj_results:
                plate = specobj_results[0]['plate']
                mjd = specobj_results[0]['mjd']
                fiberID = specobj_results[0]['fiberID']

                # 2. Now use these values to get the spectrum
                spectra = SDSS.get_spectra(plate=plate, mjd=mjd, fiberID=fiberID, data_release=17)

                if spectra:
                    spectrum = spectra[0]
                    obj_subclass = spectrum.header.get('SUBCLASS')
                    obj_class = galaxy_data['class']
                    print(f"Subclass: {obj_subclass}")
                    print(f"Source Type: {obj_class} - {obj_subclass}")
                else:
                    print("Could not retrieve spectrum.")
            else:
                print("Could not retrieve plate, mjd, fiberID.")
        else:
            print("Could not retrieve specObjID.")

    else:
        print("No matching object found in SDSS.")

except Exception as e:
    print(f"An error occurred during the SDSS query: {e}")

Redshift (z): 14.47726
Magnitudes:
u-band: 17.92454
g-band: 16.04441
r-band: 15.19806
i-band: 14.7865
z-band: 14.47726
An error occurred during the SDSS query: 'SDSSClass' object has no attribute 'query'


In [28]:
# Get a baseline set of model parameters
from prospect.models.templates import TemplateLibrary
from prospect.models import SpecModel
model_params = TemplateLibrary["parametric_sfh"]
model_params.update(TemplateLibrary["nebular"])
model_params["zred"]["init"] = obs["redshift"]

# Adjust the prior for mass, giving a wider range
from prospect.models import priors
model_params["mass"]["prior"] = priors.LogUniform(mini=1e6, maxi=1e13)

# Instantiate the model using this parameter dictionary
model = SpecModel(model_params)
assert len(model.free_params) == 5
print(model)


ModuleNotFoundError: No module named 'prospect'

In [1]:
#notes 2.26.25

#load all the data cvs from SN counts, plot cumalitive distrubition to reproduve the number of dectection ( look at the pic i took 
#work on prospector, (absolute magnitude )

#predict efficiance of the surbey of super nova 
#with qcode use to get the limiting mag of one supernoave, get the 'field', "observation date' and limmag"
